In [26]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import time, ast, os
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F


In [2]:
pd.set_option('display.max_columns', 50)
START_POI = 0
END_POI = 400
START_DATE = '2018-12-31'
TOTAL_DAYS = 400
TRAIN_RATIO = 0.5
VALID_RATIO = 0.45
TEST_RATIO = 1 - (TRAIN_RATIO + VALID_RATIO)
WINDOW_SIZE = 24
HORIZON = 6

In [3]:
class bounding_box:
    def __init__(self, _lat_min, _lon_min,_lat_max,_lon_max):
        self.lat_min = _lat_min
        self.lon_min = _lon_min
        self.lat_max = _lat_max
        self.lon_max = _lon_max


class stat_collector:
    def __init__(self):
        self.parquet_file_count=0
        self.data_record_count = 0
        self.memory_usage_in_GB = 0		#gives an estimate of the total RAM usage if all files were read into memory at the same time.
        self.unique_device_count = 0
        self.avg_pos_acc = 0
        self.starting_time = time.process_time()
        self.elapsed_time = time.process_time()
        self.unique_geohash_count = 0
        
def load_poi_db(HT_range=0.2, ht_lat=29.749907, ht_lon=-95.358421):
    HT_box = bounding_box(ht_lat - HT_range, ht_lon - HT_range, ht_lat + HT_range, ht_lon + HT_range)

    bbox = HT_box
    poi_folder = "/storage/dataset/poi_haowen/CoreRecords-CORE_POI-2019_03-2020-03-25/"
    poi_columns = ["safegraph_place_id", "parent_safegraph_place_id", "location_name", "safegraph_brand_ids", "brands",
                   "top_category", "sub_category", "naics_code", "latitude", "longitude", "street_address", "city",
                   "region", "postal_code", "iso_country_code", "phone_number", "open_hours", "category_tags"]
    files = os.listdir(poi_folder)


    poi_s = stat_collector()
    poi_db = pd.DataFrame(columns=poi_columns)
    for f in files:
        if f[-3:] == 'csv' and 'brand' not in f:
            print(f)
            df = pd.read_csv(poi_folder + f)
            df = df[
                (df['latitude'] > bbox.lat_min) & (df['latitude'] < bbox.lat_max) & (df['longitude'] > bbox.lon_min) & (
                            df['longitude'] < bbox.lon_max)]
            poi_db = poi_db.append(df, ignore_index=True, sort=False)
            poi_s.memory_usage_in_GB += df.memory_usage(deep=True).sum() / 1000000000
            poi_s.data_record_count += df.shape[0]
            poi_s.parquet_file_count += 1
    return poi_db, poi_s


def get_merged_df(csv_path, start_row, end_row):

    #start = time.time()
    merge_df = pd.read_csv(csv_path)

    merge_df = merge_df.sort_values(by=['raw_visit_counts'], ascending=False)
    merge_df = merge_df.iloc[start_row:end_row]
    #print(merge_df)
    merge_df["visits_by_each_hour"] = merge_df["visits_by_each_hour"].apply(lambda x: ast.literal_eval(x))
    merge_df["visits_by_day"] = merge_df["visits_by_day"].apply(lambda x: ast.literal_eval(x))
    merge_df["visits_by_each_hour"] = merge_df["visits_by_each_hour"].apply(lambda x: x[:TOTAL_DAYS*24])
    merge_df["visits_by_day"] = merge_df["visits_by_day"].apply(lambda x: x[:TOTAL_DAYS])
    return merge_df


In [4]:
def masked_MAPE(v, v_, axis=None):
    '''
    Mean absolute percentage error.
    :param v: np.ndarray or int, ground truth.
    :param v_: np.ndarray or int, prediction.
    :param axis: axis to do calculation.
    :return: int, MAPE averages on all elements of input.
    '''
    # TODO: Remove this line
    v_ = np.where(v_<0, 0,v_)
    
    mask = (v < 1E-5)
    percentage = np.abs(v_ - v) / np.abs(v)
    percentage = np.where(percentage > 5, 5, percentage) # TODO remove this
    if np.any(mask):
        masked_array = np.ma.masked_array(percentage, mask=mask)  # mask the dividing-zero as invalid
        result = masked_array.mean(axis=axis)
        if isinstance(result, np.ma.MaskedArray):
            return result.filled(np.nan)
        else:
            return result
    return np.mean(percentage, axis).astype(np.float64)


def MAPE(v, v_, axis=None):
    '''
    Mean absolute percentage error.
    :param v: np.ndarray or int, ground truth.
    :param v_: np.ndarray or int, prediction.
    :param axis: axis to do calculation.
    :return: int, MAPE averages on all elements of input.
    '''
    # TODO: Remove this line
    v_ = np.where(v_<0, 0,v_)
    mape = (np.abs(v_ - v) / np.abs(v)+1e-5).astype(np.float64)
    mape = np.where(mape > 5, 5, mape)
    return np.mean(mape, axis)


def RMSE(v, v_, axis=None):
    '''
    Mean squared error.
    :param v: np.ndarray or int, ground truth.
    :param v_: np.ndarray or int, prediction.
    :param axis: axis to do calculation.
    :return: int, RMSE averages on all elements of input.
    '''
    return np.sqrt(np.mean((v_ - v) ** 2, axis)).astype(np.float64)




def MAE(v, v_, axis=None):
    '''
    Mean absolute error.
    :param v: np.ndarray or int, ground truth.
    :param v_: np.ndarray or int, prediction.
    :param axis: axis to do calculation.
    :return: int, MAE averages on all elements of input.
    '''

    return np.mean(np.abs(v_ - v), axis).astype(np.float64)


def evaluate(y, y_hat, by_step=False, by_node=False):
    '''
    :param y: array in shape of [count, time_step, node].
    :param y_hat: in same shape with y.
    :param by_step: evaluate by time_step dim.
    :param by_node: evaluate by node dim.
    :return: array of mape, mae and rmse.
    '''
    if not by_step and not by_node:
        return masked_MAPE(y, y_hat), MAE(y, y_hat), RMSE(y, y_hat)
    if by_step and by_node:
        return masked_MAPE(y, y_hat, axis=0), MAE(y, y_hat, axis=0), RMSE(y, y_hat, axis=0)
    if by_step:
        return masked_MAPE(y, y_hat, axis=(0, 2)), MAE(y, y_hat, axis=(0, 2)), RMSE(y, y_hat, axis=(0, 2))
    if by_node:
        return masked_MAPE(y, y_hat, axis=(0, 1)), MAE(y, y_hat, axis=(0, 1)), RMSE(y, y_hat, axis=(0, 1))

In [5]:
csv_path_weekly = '/home/users/arash/datasets/safegraph/weekly_patterns_2018-12-31_2020-06-08_Houston.csv'
csv_poi_info = '/home/users/arash/datasets/safegraph/core_poi_info_2018-12-31_2020-06-08.csv'
weekly_patterns = get_merged_df(csv_path_weekly, START_POI, END_POI)
poi_info = pd.read_csv(csv_poi_info)
poi_df = pd.merge(weekly_patterns, poi_info, on='safegraph_place_id', how='inner')
print(poi_df.shape)
poi_df.head(3)

(400, 12)


,safegraph_place_id,visits_by_day,visits_by_each_hour,raw_visit_counts,location_name,street_address,city,region,postal_code,iso_country_code,safegraph_brand_ids,brands
0,sg:4cc165ff43ec4ce29e9dbe0732267ab1,"[6428, 6911, 8643, 8195, 7936, 7265, 8236, 614...","[74, 42, 23, 123, 220, 284, 338, 311, 493, 331...",3626878,George Bush Intercontinental Airport,2800 N Terminal Rd,Houston,TX,77032,US,NaN,NaN
1,sg:e8af4e248bbf41549aaec725d038ee42,"[2681, 3062, 3678, 3380, 3220, 3136, 3435, 255...","[23, 16, 8, 34, 72, 107, 143, 148, 212, 112, 1...",1448988,American Express Centurion Lounge,2800 N Terminal Rd Terminal D,Houston,TX,77032,US,NaN,NaN
2,sg:01cebfb757224fbd8151ee6ac6b0d679,"[3688, 3007, 3308, 3478, 3947, 4036, 2419, 212...","[8, 1, 4, 3, 5, 12, 18, 50, 61, 102, 158, 239,...",1380483,Simon mall,5085 Westheimer Rd,Houston,TX,77056,US,SG_BRAND_0a3c99595c9d3fddfece9c4e7607e5b3,Simon mall


In [6]:
poi_db, poi_s = load_poi_db(HT_range=1.5)
poi_df = poi_df.merge(poi_db, how='left', on='safegraph_place_id', suffixes=('', '_y'))
poi_df.drop(poi_df.filter(regex='_y$').columns, axis=1, inplace=True)
del poi_db
poi_df.head(3)

core_poi-part2.csv
core_poi-part5.csv
core_poi-part4.csv
core_poi-part3.csv
core_poi-part1.csv


,safegraph_place_id,visits_by_day,visits_by_each_hour,raw_visit_counts,location_name,street_address,city,region,postal_code,iso_country_code,safegraph_brand_ids,brands,parent_safegraph_place_id,top_category,sub_category,naics_code,latitude,longitude,phone_number,open_hours,category_tags
0,sg:4cc165ff43ec4ce29e9dbe0732267ab1,"[6428, 6911, 8643, 8195, 7936, 7265, 8236, 614...","[74, 42, 23, 123, 220, 284, 338, 311, 493, 331...",3626878,George Bush Intercontinental Airport,2800 N Terminal Rd,Houston,TX,77032,US,NaN,NaN,NaN,Support Activities for Air Transportation,Other Airport Operations,488119.0,29.981382,-95.322839,NaN,NaN,NaN
1,sg:e8af4e248bbf41549aaec725d038ee42,"[2681, 3062, 3678, 3380, 3220, 3136, 3435, 255...","[23, 16, 8, 34, 72, 107, 143, 148, 212, 112, 1...",1448988,American Express Centurion Lounge,2800 N Terminal Rd Terminal D,Houston,TX,77032,US,NaN,NaN,sg:f4a955def8ca49fd87153af82a237245,Gasoline Stations,Gasoline Stations with Convenience Stores,447110.0,29.987829,-95.334916,NaN,"{ ""Mon"": [[""5:30"", ""21:00""]], ""Tue"": [[""5:30"",...",NaN
2,sg:01cebfb757224fbd8151ee6ac6b0d679,"[3688, 3007, 3308, 3478, 3947, 4036, 2419, 212...","[8, 1, 4, 3, 5, 12, 18, 50, 61, 102, 158, 239,...",1380483,Simon mall,5085 Westheimer Rd,Houston,TX,77056,US,SG_BRAND_0a3c99595c9d3fddfece9c4e7607e5b3,Simon mall,NaN,Lessors of Real Estate,Malls,531120.0,29.738954,-95.463803,NaN,"{ ""Mon"": [[""10:00"", ""19:00""]], ""Tue"": [[""10:00...",NaN


In [7]:
poi_df['top_category'].map({'Support Activities for Air Transportation':1.5}).fillna(2)

0      1.5
1      2.0
2      2.0
3      2.0
4      1.5
      ... 
395    2.0
396    2.0
397    2.0
398    2.0
399    2.0
Name: top_category, Length: 400, dtype: float64

In [8]:
poi_df.head(3)

,safegraph_place_id,visits_by_day,visits_by_each_hour,raw_visit_counts,location_name,street_address,city,region,postal_code,iso_country_code,safegraph_brand_ids,brands,parent_safegraph_place_id,top_category,sub_category,naics_code,latitude,longitude,phone_number,open_hours,category_tags
0,sg:4cc165ff43ec4ce29e9dbe0732267ab1,"[6428, 6911, 8643, 8195, 7936, 7265, 8236, 614...","[74, 42, 23, 123, 220, 284, 338, 311, 493, 331...",3626878,George Bush Intercontinental Airport,2800 N Terminal Rd,Houston,TX,77032,US,NaN,NaN,NaN,Support Activities for Air Transportation,Other Airport Operations,488119.0,29.981382,-95.322839,NaN,NaN,NaN
1,sg:e8af4e248bbf41549aaec725d038ee42,"[2681, 3062, 3678, 3380, 3220, 3136, 3435, 255...","[23, 16, 8, 34, 72, 107, 143, 148, 212, 112, 1...",1448988,American Express Centurion Lounge,2800 N Terminal Rd Terminal D,Houston,TX,77032,US,NaN,NaN,sg:f4a955def8ca49fd87153af82a237245,Gasoline Stations,Gasoline Stations with Convenience Stores,447110.0,29.987829,-95.334916,NaN,"{ ""Mon"": [[""5:30"", ""21:00""]], ""Tue"": [[""5:30"",...",NaN
2,sg:01cebfb757224fbd8151ee6ac6b0d679,"[3688, 3007, 3308, 3478, 3947, 4036, 2419, 212...","[8, 1, 4, 3, 5, 12, 18, 50, 61, 102, 158, 239,...",1380483,Simon mall,5085 Westheimer Rd,Houston,TX,77056,US,SG_BRAND_0a3c99595c9d3fddfece9c4e7607e5b3,Simon mall,NaN,Lessors of Real Estate,Malls,531120.0,29.738954,-95.463803,NaN,"{ ""Mon"": [[""10:00"", ""19:00""]], ""Tue"": [[""10:00...",NaN


In [9]:
cat_cols = ['top_category', 'sub_category', 'postal_code']

In [10]:
cat_col = poi_df['sub_category'].astype('category')

In [11]:
cat_code_dict = {}
for col in cat_cols:
    category_col = poi_df[col].astype('category')
    cat_code_dict[col] = {value: idx for idx, value in enumerate(category_col.cat.categories)} 
cat_code_dict

{'top_category': {'Amusement Parks and Arcades': 0,
  'Automobile Dealers': 1,
  'Automotive Equipment Rental and Leasing': 2,
  'Book Stores and News Dealers': 3,
  'Building Material and Supplies Dealers': 4,
  'Child Day Care Services': 5,
  'Clothing Stores': 6,
  'Colleges, Universities, and Professional Schools': 7,
  'Continuing Care Retirement Communities and Assisted Living Facilities for the Elderly': 8,
  'Department Stores': 9,
  'Depository Credit Intermediation': 10,
  'Educational Support Services': 11,
  'Elementary and Secondary Schools': 12,
  'Florists': 13,
  'Furniture Stores': 14,
  'Gambling Industries': 15,
  'Gasoline Stations': 16,
  'General Medical and Surgical Hospitals': 17,
  'General Merchandise Stores, including Warehouse Clubs and Supercenters': 18,
  'Grocery Stores': 19,
  'Health and Personal Care Stores': 20,
  'Home Health Care Services': 21,
  'Jewelry, Luggage, and Leather Goods Stores': 22,
  'Junior Colleges': 23,
  'Justice, Public Order, and

In [12]:
# categorical fields
for col in cat_cols:
    code_dict = cat_code_dict[col]
    code_fillna_value = len(code_dict)
    poi_df[col+'_cat'] = poi_df[col].map(code_dict).fillna(code_fillna_value).astype(np.int64)
poi_df.head(3)

,safegraph_place_id,visits_by_day,visits_by_each_hour,raw_visit_counts,location_name,street_address,city,region,postal_code,iso_country_code,safegraph_brand_ids,brands,parent_safegraph_place_id,top_category,sub_category,naics_code,latitude,longitude,phone_number,open_hours,category_tags,top_category_cat,sub_category_cat,postal_code_cat
0,sg:4cc165ff43ec4ce29e9dbe0732267ab1,"[6428, 6911, 8643, 8195, 7936, 7265, 8236, 614...","[74, 42, 23, 123, 220, 284, 338, 311, 493, 331...",3626878,George Bush Intercontinental Airport,2800 N Terminal Rd,Houston,TX,77032,US,NaN,NaN,NaN,Support Activities for Air Transportation,Other Airport Operations,488119.0,29.981382,-95.322839,NaN,NaN,NaN,39,39,21
1,sg:e8af4e248bbf41549aaec725d038ee42,"[2681, 3062, 3678, 3380, 3220, 3136, 3435, 255...","[23, 16, 8, 34, 72, 107, 143, 148, 212, 112, 1...",1448988,American Express Centurion Lounge,2800 N Terminal Rd Terminal D,Houston,TX,77032,US,NaN,NaN,sg:f4a955def8ca49fd87153af82a237245,Gasoline Stations,Gasoline Stations with Convenience Stores,447110.0,29.987829,-95.334916,NaN,"{ ""Mon"": [[""5:30"", ""21:00""]], ""Tue"": [[""5:30"",...",NaN,16,22,21
2,sg:01cebfb757224fbd8151ee6ac6b0d679,"[3688, 3007, 3308, 3478, 3947, 4036, 2419, 212...","[8, 1, 4, 3, 5, 12, 18, 50, 61, 102, 158, 239,...",1380483,Simon mall,5085 Westheimer Rd,Houston,TX,77056,US,SG_BRAND_0a3c99595c9d3fddfece9c4e7607e5b3,Simon mall,NaN,Lessors of Real Estate,Malls,531120.0,29.738954,-95.463803,NaN,"{ ""Mon"": [[""10:00"", ""19:00""]], ""Tue"": [[""10:00...",NaN,25,34,38


In [13]:
poi_df['top_category_cat'].to_numpy().shape

(400,)

In [14]:
torch.tensor(poi_df['top_category_cat'].to_numpy())

tensor([39, 16, 25, 41, 39, 25, 39, 39, 39, 12, 39, 25, 25, 25, 27, 37, 17, 11,
        25, 25, 35, 25, 35, 25, 42, 35,  7, 25, 25, 17, 17, 25, 12, 25, 25,  7,
        35, 25,  7, 25, 17, 42, 25, 25, 18, 18, 25, 25, 12, 25, 35, 25, 12, 25,
        25, 25, 23, 12, 25, 18, 23, 12, 25, 27, 29, 18, 25, 19, 30, 18, 25, 25,
        18, 18, 35, 18, 18, 27, 25, 25, 18, 18, 25, 16, 18, 25, 12, 37, 27, 25,
        25,  3, 19, 25, 25, 25,  8, 18, 18, 41, 25, 12, 25, 25,  5, 25, 25, 17,
        18, 25, 19, 25, 18, 35, 25, 41, 25, 25, 25, 35, 42, 25, 19, 23, 18, 25,
        25, 16,  4, 25,  9, 14, 25, 18, 35, 35, 20, 17, 25, 27, 41, 39, 12, 25,
        35, 25, 12, 25, 29, 18,  5, 25, 35, 18, 12, 35, 14, 25, 25, 31, 25, 21,
        23, 25, 35, 30, 30, 25, 25, 25, 25, 42, 42, 18, 25, 17, 15, 18, 12, 30,
        25, 42, 18,  7, 16,  1, 35, 41, 25, 25, 25, 25,  6, 12, 25, 25,  5, 35,
        36, 25, 18, 35, 12, 37, 25,  9, 41, 25, 16, 19, 25, 25, 25, 12, 25, 35,
         4, 29, 25,  9, 29, 27, 27, 29, 

In [27]:
one_hot_arr = F.one_hot(torch.tensor(poi_df['top_category_cat'].to_numpy()))
one_hot_arr

tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 1, 0,  ..., 0, 0, 0]])

In [16]:
torch.argmax(one_hot_arr, dim=1)

tensor([39, 16, 25, 41, 39, 25, 39, 39, 39, 12, 39, 25, 25, 25, 27, 37, 17, 11,
        25, 25, 35, 25, 35, 25, 42, 35,  7, 25, 25, 17, 17, 25, 12, 25, 25,  7,
        35, 25,  7, 25, 17, 42, 25, 25, 18, 18, 25, 25, 12, 25, 35, 25, 12, 25,
        25, 25, 23, 12, 25, 18, 23, 12, 25, 27, 29, 18, 25, 19, 30, 18, 25, 25,
        18, 18, 35, 18, 18, 27, 25, 25, 18, 18, 25, 16, 18, 25, 12, 37, 27, 25,
        25,  3, 19, 25, 25, 25,  8, 18, 18, 41, 25, 12, 25, 25,  5, 25, 25, 17,
        18, 25, 19, 25, 18, 35, 25, 41, 25, 25, 25, 35, 42, 25, 19, 23, 18, 25,
        25, 16,  4, 25,  9, 14, 25, 18, 35, 35, 20, 17, 25, 27, 41, 39, 12, 25,
        35, 25, 12, 25, 29, 18,  5, 25, 35, 18, 12, 35, 14, 25, 25, 31, 25, 21,
        23, 25, 35, 30, 30, 25, 25, 25, 25, 42, 42, 18, 25, 17, 15, 18, 12, 30,
        25, 42, 18,  7, 16,  1, 35, 41, 25, 25, 25, 25,  6, 12, 25, 25,  5, 35,
        36, 25, 18, 35, 12, 37, 25,  9, 41, 25, 16, 19, 25, 25, 25, 12, 25, 35,
         4, 29, 25,  9, 29, 27, 27, 29, 

In [18]:
all(torch.tensor(poi_df['top_category_cat'].to_numpy()) == \
torch.argmax(one_hot_arr, dim=1))


True

In [28]:
poi_df[['top_category_cat', 'sub_category_cat']].to_numpy()

array([[39, 39],
       [16, 22],
       [25, 34],
       [41, 29],
       [39, 39],
       [25, 34],
       [39, 39],
       [39, 39],
       [39, 39],
       [12, 16],
       [39, 39],
       [25, 34],
       [25, 34],
       [25, 34],
       [27, 27],
       [37, 49],
       [17, 23],
       [11, 15],
       [25, 34],
       [25, 34],
       [35, 20],
       [25, 34],
       [35, 33],
       [25, 34],
       [42, 51],
       [35, 47],
       [ 7,  9],
       [25, 34],
       [25, 34],
       [17, 23],
       [17, 23],
       [25, 34],
       [12, 16],
       [25, 34],
       [25, 34],
       [ 7,  9],
       [35, 20],
       [25, 34],
       [ 7,  9],
       [25, 34],
       [17, 23],
       [42, 51],
       [25, 34],
       [25, 34],
       [18,  1],
       [18,  1],
       [25, 34],
       [25, 34],
       [12, 16],
       [25, 34],
       [35, 47],
       [25, 34],
       [12, 16],
       [25, 34],
       [25, 34],
       [25, 34],
       [23, 31],
       [12, 16],
       [25, 34

In [32]:
arr1 = F.one_hot(torch.tensor(poi_df['top_category_cat'].to_numpy()))
arr2 = F.one_hot(torch.tensor(poi_df['sub_category_cat'].to_numpy()))
print(arr1.shape, arr2.shape)

torch.Size([400, 43]) torch.Size([400, 52])


In [47]:
torch.cat([arr1, arr2], dim=1)[0, 40:].argmax()

tensor(42)

In [43]:
poi_df.head(1)

,safegraph_place_id,visits_by_day,visits_by_each_hour,raw_visit_counts,location_name,street_address,city,region,postal_code,iso_country_code,safegraph_brand_ids,brands,parent_safegraph_place_id,top_category,sub_category,naics_code,latitude,longitude,phone_number,open_hours,category_tags,top_category_cat,sub_category_cat,postal_code_cat
0,sg:4cc165ff43ec4ce29e9dbe0732267ab1,"[6428, 6911, 8643, 8195, 7936, 7265, 8236, 614...","[74, 42, 23, 123, 220, 284, 338, 311, 493, 331...",3626878,George Bush Intercontinental Airport,2800 N Terminal Rd,Houston,TX,77032,US,NaN,NaN,NaN,Support Activities for Air Transportation,Other Airport Operations,488119.0,29.981382,-95.322839,NaN,NaN,NaN,39,39,21


In [48]:
cat_code_dict['sub_category']

{'All Other Amusement and Recreation Industries': 0,
 'All Other General Merchandise Stores': 1,
 'Amusement and Theme Parks': 2,
 'Assisted Living Facilities for the Elderly': 3,
 'Book Stores': 4,
 'Bowling Centers': 5,
 'Casinos (except Casino Hotels)': 6,
 'Child Day Care Services': 7,
 "Children's and Infants' Clothing Stores": 8,
 'Colleges, Universities, and Professional Schools': 9,
 'Commercial Banking': 10,
 'Confectionery and Nut Stores': 11,
 'Correctional Institutions': 12,
 'Cosmetics, Beauty Supplies, and Perfume Stores': 13,
 'Department Stores': 14,
 'Educational Support Services': 15,
 'Elementary and Secondary Schools': 16,
 'Family Clothing Stores': 17,
 'Fitness and Recreational Sports Centers': 18,
 'Florists': 19,
 'Full-Service Restaurants': 20,
 'Furniture Stores': 21,
 'Gasoline Stations with Convenience Stores': 22,
 'General Medical and Surgical Hospitals': 23,
 'Gift, Novelty, and Souvenir Stores': 24,
 'Golf Courses and Country Clubs': 25,
 'Hardware Store